In [21]:
import pandas as pd
from pymatgen.core import Structure
from pymatgen.util.testing import PymatgenTest

from lematerial_forgebench.benchmarks.stability_benchmark import StabilityBenchmark
from lematerial_forgebench.benchmarks.validity_benchmark import ValidityBenchmark
from lematerial_forgebench.metrics.stability_metrics import StabilityMetric
from lematerial_forgebench.preprocess.reference_energies import (
    get_energy_above_hull,
    get_formation_energy_from_composition_energy,
)
from lematerial_forgebench.preprocess.stability_preprocess import (
    EnergyAboveHull,
    OrbFormationEnergy,
    StabilityPreprocessor,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
def test_stability_metric(test_structures):
    metric = StabilityMetric(
        relaxer_type="orb",
        # relaxer_config={"steps": 500, "fmax": 0.02},
    )

    result = metric(test_structures)

    # Check that computation ran (may have failed indices due to no MP entries)
    assert len(result.individual_values) == len(test_structures)

    # Check result structure
    print(result.metrics)
    assert "mean_e_above_hull" in result.metrics
    assert "stable_ratio" in result.metrics
    assert result.primary_metric == "stable_ratio"

    assert result.metrics["stable_ratio"] >= 0.0

In [32]:
test_stability_metric(structures)

{'stable_ratio': np.float64(0.5), 'mean_e_above_hull': np.float64(0.025)}


In [23]:
test = PymatgenTest()
structures = [
    test.get_structure("Si"),
    test.get_structure("LiFePO4"),
]

# Add precomputed e_above_hull values to structure properties
e_above_hull_values = [0.05, 0.0]  # Mix of stable and metastable

for structure, e_above_hull in zip(structures, e_above_hull_values):
    structure.properties["e_above_hull"] = e_above_hull


C:\Users\samue\AppData\Local\Temp\ipykernel_45704\2378555641.py:1: FutureWarning: PymatgenTest is deprecated, and will be removed on 2026-01-01
Use MatSciTest in pymatgen.util.testing instead.
  test = PymatgenTest()


In [25]:
metric = StabilityMetric(
    relaxer_type="orb",
    # relaxer_config={"steps": 500, "fmax": 0.02},
)

result = metric(structures)

In [28]:
result

MetricResult(metrics={'stable_ratio': np.float64(0.5), 'mean_e_above_hull': np.float64(0.025)}, primary_metric='stable_ratio', uncertainties={'e_above_hull_std': {'std': np.float64(0.025)}}, config=MetricConfig(name='StabilityMetric', description='Evaluates structure stability from precomputed e_above_hull', lower_is_better=False, n_jobs=1), computation_time=0.0, n_structures=2, individual_values=[0.05, 0.0], failed_indices=[], warnings=[])

In [27]:
assert "mean_e_above_hull" in result.metrics
assert "stable_ratio" in result.metrics
assert "metastable_ratio" in result.metrics
assert result.primary_metric == "stable_ratio"

AssertionError: 

In [10]:
test = PymatgenTest()

filename = 'CsBr.cif'
structure = Structure.from_file(filename)
structure = structure.remove_oxidation_states()

filename2 = 'CsPbBr3.cif'
structure2 = Structure.from_file(filename2)
structure2 = structure2.remove_oxidation_states()

structures = [structure, structure2, test.get_structure("Si"), test.get_structure("LiFePO4")]

C:\Users\samue\AppData\Local\Temp\ipykernel_45704\2301446376.py:1: FutureWarning: PymatgenTest is deprecated, and will be removed on 2026-01-01
Use MatSciTest in pymatgen.util.testing instead.
  test = PymatgenTest()


In [18]:
benchmark = ValidityBenchmark()
benchmark_result = benchmark.evaluate(structures) 

C:\Users\samue\lematerial-forgebench\.venv\Lib\site-packages\pymatgen\analysis\local_env.py:4022: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r1 = _get_radius(structure[n])
C:\Users\samue\lematerial-forgebench\.venv\Lib\site-packages\pymatgen\analysis\local_env.py:4024: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r2 = _get_radius(entry["site"])
C:\Users\samue\lematerial-forgebench\.venv\Lib\site-packages\pymatgen\analysis\local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  nn_data = self.get_nn_data(structure, n)
C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\validity_metrics.py:882: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifd

MetricResult(metrics={'validity_score': np.float64(0.6125), 'valid_structures_ratio': 0.0}, primary_metric='validity_score', uncertainties={'validity_score': {'std': np.float64(0.1030776406404415)}}, config=CompositeValidityConfig(name='CompositeValidity', description='Combined measure of structure validity', lower_is_better=False, n_jobs=1, metrics={'charge_neutrality': ChargeNeutralityConfig(name='ChargeNeutrality', description='Measures how close a structure is to charge neutrality', lower_is_better=True, n_jobs=1, tolerance=0.1, strict=False), 'min_distance': MinimumInteratomicDistanceConfig(name='MinimumInteratomicDistance', description='Evaluates if interatomic distances are physically reasonable', lower_is_better=False, n_jobs=1, element_radii=None, scaling_factor=0.5), 'coordination': CoordinationEnvironmentConfig(name='CoordinationEnvironment', description='Evaluates if coordination environments match chemical expectations', lower_is_better=False, n_jobs=1, environment_databas

C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\validity_metrics.py:882: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Cs0', 'Cs0', 'Cs0', 'Cs0', 'Br1', 'Br1', 'Br1', 'Br1']`.
  cif_writer = CifWriter(structure)
C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\validity_metrics.py:882: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Cs0', 'Cs0', 'Cs0', 'Cs0', 'Pb1', 'Pb1', 'Pb1', 'Pb1', 'Br2', 'Br2', 'Br2', 'Br2', 'Br3', 'Br3', 'Br3', 'Br3', 'Br4', 'Br4', 'Br4', 'Br4']`.
  cif_writer = CifWriter(structure)


In [19]:
benchmark_result

BenchmarkResult(evaluator_results={'charge_neutrality': {'combined_value': np.float64(0.0), 'charge_neutrality_value': np.float64(0.0), 'metric_results': {'charge_neutrality': MetricResult(metrics={'charge_neutrality_error': np.float64(0.0), 'charge_neutral_ratio': 1.0}, primary_metric='charge_neutrality_error', uncertainties={'charge_neutrality_error': {'std': np.float64(0.0)}}, config=ChargeNeutralityConfig(name='ChargeNeutrality', description='Measures how close a structure is to charge neutrality', lower_is_better=True, n_jobs=1, tolerance=0.1, strict=False), computation_time=0.09488987922668457, n_structures=4, individual_values=[0.0, 0.0, 0.0, 0.0], failed_indices=[], warnings=[])}}, 'interatomic_distance': {'combined_value': np.float64(1.0), 'min_distance_value': np.float64(1.0), 'metric_results': {'min_distance': MetricResult(metrics={'min_distance_score': np.float64(1.0), 'valid_structures_ratio': 1.0}, primary_metric='min_distance_score', uncertainties={'min_distance_score': 

C:\Users\samue\AppData\Local\Temp\ipykernel_45704\2301446376.py:1: FutureWarning: PymatgenTest is deprecated, and will be removed on 2026-01-01
Use MatSciTest in pymatgen.util.testing instead.
  test = PymatgenTest()


In [4]:
stability_preprocessor = StabilityPreprocessor()
preprocessor_result = stability_preprocessor(structures)

cpu


C:\Users\samue\lematerial-forgebench\.venv\Lib\site-packages\orb_models\utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


energy_above_hull relaxed : 0.0


C:\Users\samue\lematerial-forgebench\.venv\Lib\site-packages\orb_models\utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


energy_above_hull relaxed : 0.028652646023437534
energy_above_hull relaxed : 0.009661548326415748
energy_above_hull relaxed : 0.24249804714564505


In [5]:
benchmark = StabilityBenchmark()
benchmark_result = benchmark.evaluate(preprocessor_result.processed_structures)

Cs4 Br4 Formation Energy : -7.149936223230794
Cs4 Pb4 Br12 Formation Energy : -8.492938243082682
Si2 Formation Energy : 0.02191613711914009
Li4 Fe4 P4 O16 Formation Energy : -12.061907388726752
Cs4 Br4 Relaxation Stability RMSE : 0.03093874998560804
Cs4 Pb4 Br12 Relaxation Stability RMSE : 0.18214181382134303
Si2 Relaxation Stability RMSE : 0.022634037275015083
Li4 Fe4 P4 O16 Relaxation Stability RMSE : 0.05637589515774106
{'stable_ratio': 0.25, 'metastable_ratio': 0.75, 'mean_e_above_hull': 0.07161587097197553, 'mean_formation_energy': -6.920716429480272, 'mean_relaxation_RMSE': 0.07302262405992681}


In [7]:
benchmark_result.final_scores

{'stable_ratio': 0.25,
 'metastable_ratio': 0.75,
 'mean_e_above_hull': 0.07161587097197553,
 'mean_formation_energy': -6.920716429480272,
 'mean_relaxation_RMSE': 0.07302262405992681}